## <center> PROJECT. Определяем количество ботов на сайте

### Цели
1. Оценить количество ботов, которые заходят на наш сайт, и их долю от всего трафика.
2. Определить, с каких источников и в какое время суток чаще всего заходят роботы.

In [12]:
# загрузим необходимые библиотеки и функции
import json 
import requests
from pprint import pprint
import pandas as pd 
# вводные для яндекс.метрики
token = 'AQAAAAAB95q6AAfACLdBhhkWNkBjiL0XU9oCf-o' 
headers = {'Authorization': 'OAuth ' + token}
# получение данных из Яндекс.Метрики
## isRobot - Посетитель предположительно является роботом,
## ym:s:isRobot - Доля посетителей, предположительно являющихся роботами.
params = {'metrics': 'ym:s:visits,ym:s:robotPercentage',
          'dimensions':'ym:s:lastsignTrafficSource,ym:s:hour,ym:s:isRobot', # атрибуция по последнему значимому источнику, 
          'date1': '2020-09-01',
          'date2': '2020-09-30',
          'ids': 30177909,
          'accuracy':'full',
          'limit':100000}
# Напишем строку кода, при выполнении которой будет сделан GET-запрос к ресурсу
response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers) 
# print (response.status_code) # проверка доступности - код 200
metrika_data = response.json() # преоброзование полученных данных в словарь с помощью метода json()

# Преобразуем данные. Избавляемся от вложенных словарей.
def getMetrikaDataInListOfDicts(metrika_data):
    list_of_dicts = []
    dimensions_list = metrika_data['query']['dimensions']
    metrics_list = metrika_data['query']['metrics']
    for data_item in metrika_data['data']:
        d = {}
        for i,dimension in enumerate(data_item['dimensions']):
            d[dimensions_list[i]] = dimension['name']
        for i,metric in enumerate(data_item['metrics']):
            d[metrics_list[i]] = metric
        list_of_dicts.append(d)
    return list_of_dicts
metrika_list_of_dicts = getMetrikaDataInListOfDicts(metrika_data)
# создаем датафрейм 
metrika_df = pd.DataFrame(metrika_list_of_dicts)

# Для удобства переименуем столбцы 
metrika_df.columns=['source','visit_time','people_robot','visits','robot_visits%']

In [13]:
# проверим датафрейм на пропуски
metrika_df.isnull().sum()

source           0
visit_time       0
people_robot     0
visits           0
robot_visits%    0
dtype: int64

Пропусков нет, поэтому переходим к следующему шагу

In [14]:
# Цель 1. Считаем количество роботов и их долю
pt_robot_visits = metrika_df.pivot_table(values = ['visits'], columns = ['people_robot'], aggfunc = 'sum')
pt_robot_visits['robot_visits%'] = round(pt_robot_visits['Robots'] * 100 / (pt_robot_visits['Robots'] + pt_robot_visits['People']), 2)
display(pt_robot_visits)

people_robot,People,Robots,robot_visits%
visits,3353.0,962.0,22.29


In [15]:
# Цель 2. Определяем, с каких источников и в какое время суток чаще всего заходят роботы.

### 1 part. Общая сводная таблица, где увидим All - кол-во роботов по источнику
pt_robot_source_time = metrika_df.loc[metrika_df['people_robot'].isin(['Robots'])].pivot_table(values = ['visits'],
                                              index = ['visit_time'],
                                              columns = ['source'],
                                              aggfunc = 'sum',
                                              fill_value = 0,
                                              margins = True)
display(pt_robot_source_time)

visits                                               \
source     Cached page traffic Direct traffic Internal traffic Link traffic   
visit_time                                                                    
00:00                        0              2                0            0   
01:00                        1              4                0            0   
02:00                        0              2                0            0   
03:00                        0              2                0            1   
04:00                        0              3                0            0   
05:00                        0              3                0            0   
06:00                        0              3                0            2   
07:00                        0              5                0            1   
08:00                        0              5                0            1   
09:00                        0             12                1            1   
10:00                        0              9                0            6   
11:00                        0              8                0            3   
12:00                        0             15                1            2   
13:00                        0             13                0            1   
14:00                        0             14                0            2   
15:00                        0              6                0            1   
16:00                        0              8                1            3   
17:00                        0              9                0            2   
18:00                        0              9                0            1   
19:00                        0             10                0            0   
20:00                        0              8                0            2   
21:00                        0             13                0            0   
22:00                        0              6                0            1   
23:00                        0              8                0            1   
All                          1            177                3           31   

                                                                
source     Search engine traffic Social network traffic    All  
visit_time                                                      
00:00                         17                      0   19.0  
01:00                          8                      0   13.0  
02:00                          5                      0    7.0  
03:00                          3                      0    6.0  
04:00                          3                      0    6.0  
05:00                          1                      0    4.0  
06:00                          3                      0    8.0  
07:00                          6                      0   12.0  
08:00                         29                      1   36.0  
09:00                         36                      1   51.0  
10:00                         36                      0   51.0  
11:00                         59                      0   70.0  
12:00                         56                      1   75.0  
13:00                         64                      2   80.0  
14:00                         58                      0   74.0  
15:00                         63                      1   71.0  
16:00                         62                      1   75.0  
17:00                         55                      0   66.0  
18:00                         31                      0   41.0  
19:00                         38                      0   48.0  
20:00                         26                      0   36.0  
21:00                         29                      1   43.0  
22:00                         30                      3   40.0  
23:00                         21                      0   30.0  
All                          739                     11 

In [16]:
### 2 part. Выводим время Search engine traffic 
pt_robot_source_search = pt_robot_source_time['visits']['Search engine traffic'].sort_values(ascending=False)
display(pt_robot_source_search)

visit_time
All      739
13:00     64
15:00     63
16:00     62
11:00     59
14:00     58
12:00     56
17:00     55
19:00     38
09:00     36
10:00     36
18:00     31
22:00     30
21:00     29
08:00     29
20:00     26
23:00     21
00:00     17
01:00      8
07:00      6
02:00      5
03:00      3
04:00      3
06:00      3
05:00      1
Name: Search engine traffic, dtype: int64

### Выводы  
* Всего роботов 962, их доля от людей составляет 22,29%  
* Самый популярный источник Search engine traffic, с этого источника заходит 739 роботов. Т.о. понимаем, что роботы генерируеют больше трафика, чем реальный пользователь. Самое популярное время для роботов как видно из сводной таблицы 13:00

